#Run this before starting
!python -c "import torch; print(torch.__version__)"
!python -c "import torch; print(torch.version.cuda)"


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from torch.nn import Parameter
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.utils import remove_self_loops, add_self_loops, softmax
import torch.optim as optim
from torch_geometric.nn.conv import feast_conv


In [2]:
import datetime
versj = 1
today = datetime.date.today().strftime("%b-%d")
print(today)

May-19


In [49]:
versj =2

In [3]:
if torch.cuda.is_available():
    device_name = torch.device("cuda")
else:
    device_name = torch.device('cpu')

print("Using {}.".format(device_name))

Using cpu.


In [4]:
#Test for new dataset
def testFinal(model, indata, device):
    model.eval()
    title = ''
    E_A = 0
    E_A_Ad = 0
    E_A_Ad2 = 0
    with torch.no_grad():
        for id, data in enumerate(indata):
          out = model(data.to(device))
          real = data.y.to(device)
          errorList = []
          error = 0
          epsilon = 1e-8
          for i in range(len(out)):
            e = ((out[i].item() - real[i].item() ) / (epsilon + out[i].item()))*100
            errorList.append(e)

          errorListAbs = [abs(x) for x in errorList]

          errorListAbsAd2 = errorListAbs[8:12]

          errorListAbs.sort()
          errorListAbsAd = errorListAbs[:-2]

          E_A_el = sum(errorListAbs)/len(errorListAbs)
          E_A_Ad_el = sum(errorListAbsAd)/len(errorListAbsAd)
          E_A_Ad_el2 = sum(errorListAbsAd2)/len(errorListAbsAd2)

          E_A += E_A_el
          E_A_Ad += E_A_Ad_el
          E_A_Ad2 += E_A_Ad_el2
          
          print([round(x,2) for x in errorList])
          print('Avarage Error: '+ str(round(E_A_el,2))+'%, Adjusted Avarage Error: '+ str(round(E_A_Ad_el,2))+'%')
          print('Mean Square: '+ str(round(F.mse_loss(out, data.y.to(device)).item(),4))+'\n')

          #errorAvg += sum(errorListAbs)/len(errorListAbs)

    print('Avarage Tot. Error: '+ str(round(E_A/len(indata),2))+'%\n')
    print('After removing the two worst errors. Avarage Adjusted Tot. Error: '+ str(round(E_A_Ad/len(indata),2))+'%')
    print('Total error for nodes 8, 9, 10, 11. Avarage Adjusted Tot. Error: '+ str(round(E_A_Ad2/len(indata),2))+'%')
   
#Test for new dataline

def testData(tester_model, datasetTest,line):
    real = datasetTest[line].y
    
    out = tester_model(datasetTest[line].to(device_name))
    Bool = ''
    print('{:<10}{:<10}{:<10}{:<10}'.format('real', 'out', '%error', 'Sign'))
    for i in range(len(out)):
        error = ((out[i].item() - real[i].item())/ out[i] )*100
        if torch.sign(real[i]) == torch.sign(out[i]):
            Bool = 'True'
        else:
            Bool = 'False'
        real_formatted = '{:<10.3f}'.format(round(real[i].item(),5))
        out_formatted = '{:<10.3f}'.format(round(out[i].item(),5))
        error_formatted = '{:<10.2f}'.format(error)
        list_formatted = [real_formatted, out_formatted, error_formatted, Bool]
        print('{:<10}{:<10}{:<10}{:<10}'.format(*list_formatted))

In [6]:
#Traindata

import numpy as np
import torch
import random
from torch_geometric.data import Data, DataLoader

#Import data function
def get_data(filepath):
    data = np.loadtxt(filepath)
    x = data[:, :20]
    z = data[:, 20:40]
    m = data[:, 40:60]

    dataX = []
    for i in range(x.shape[0]):
        dataXs = np.column_stack((np.arange(20), x[i], z[i]))
        dataX.append(dataXs)

    dataY = m
    #zipped = list(zip(dataX, dataY))
    #np.random.shuffle(zipped)
    #dataX, dataY = zip(*zipped)

    dataX = np.array(dataX)
    dataY = np.array(dataY)

    dataEdgeIndex = np.column_stack((np.arange(20)[:-1], np.arange(20)[1:]))
    dataEdgeIndex = np.vstack((dataEdgeIndex, dataEdgeIndex[:, ::-1])).T

    dataset = [Data(x=torch.from_numpy(x).float(), edge_index=torch.from_numpy(dataEdgeIndex).long(), y=torch.from_numpy(y).float()) for x, y in zip(dataX, dataY)]

    return dataset

#Getting data
datasetTrain = get_data("Data\\1DatasetTrain.txt")
datasetTest = get_data("Data\\1DatasetTest.txt")
datasetNew = get_data("Data\\1DatasetNew.txt")

In [7]:
datasetTrain4 = get_data("4DatasetTrain.txt")
datasetNew4 = get_data("4DatasetNew.txt")
datasetTest4 = get_data("4DatasetTest.txt")
datasetTest2 = get_data("data2test.txt")

FileNotFoundError: 4DatasetTrain.txt not found.

In [9]:
print(len(datasetNew))
print(len(datasetTest))

#Splitting dataset

train_loader = datasetTrain[:1000]
test_loader = datasetTrain[-250:]

print(len(train_loader))
print(len(test_loader))

print(datasetTrain[10].edge_index.shape)
print(datasetTrain[10].x.shape)
print(datasetTrain[10].y.shape)

100
1001
1000
250
torch.Size([2, 38])
torch.Size([20, 3])
torch.Size([20])


In [12]:
class ArchNN(torch.nn.Module):
    def __init__(self, in_channels, num_classes, heads, t_inv = True):
        super(ArchNN, self).__init__()
        self.fc0 = nn.Linear(in_channels, 16)
        self.conv1 = feast_conv.FeaStConv(16, 32, heads=heads, t_inv=t_inv)
        self.conv2 = feast_conv.FeaStConv(32, 64, heads=heads, t_inv=t_inv)
        self.conv3 = feast_conv.FeaStConv(64, 128, heads=heads, t_inv=t_inv)
        self.conv4 = feast_conv.FeaStConv(128, 256, heads=heads, t_inv=t_inv)
        self.conv5 = feast_conv.FeaStConv(256, 128, heads=heads, t_inv=t_inv)
        self.fc1 = nn.Linear(128, 64)
        self.fc2 = nn.Linear(64, 1)

        self.reset_parameters()

    def reset_parameters(self):
        self.conv1.reset_parameters()
        self.conv2.reset_parameters()
        self.conv3.reset_parameters()
        self.conv4.reset_parameters()
        self.conv5.reset_parameters()

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.leaky_relu(self.fc0(x))
        x = F.leaky_relu(self.conv1(x, edge_index))
        x = F.leaky_relu(self.conv2(x, edge_index))
        x = F.leaky_relu(self.conv3(x, edge_index))
        x = F.leaky_relu(self.conv4(x, edge_index))
        x = F.leaky_relu(self.conv5(x, edge_index))
        x = F.leaky_relu(self.fc1(x))
        x = self.fc2(x)
        x = torch.squeeze(x, dim=1)
        return x

class ArchNN2(torch.nn.Module):
    def __init__(self, in_channels, num_classes, heads, t_inv = True):
        super(ArchNN2, self).__init__()
        self.fc0 = nn.Linear(in_channels, 16)
        self.conv1 = feast_conv.FeaStConv(16, 32, heads=heads, t_inv=t_inv)
        self.conv2 = feast_conv.FeaStConv(32, 64, heads=heads, t_inv=t_inv)
        self.conv3 = feast_conv.FeaStConv(64, 128, heads=heads, t_inv=t_inv)
        self.conv4 = feast_conv.FeaStConv(128, 256, heads=heads, t_inv=t_inv)
        self.conv5 = feast_conv.FeaStConv(256, 128, heads=heads, t_inv=t_inv)
        self.fc1 = nn.Linear(128, 64)
        self.fc2 = nn.Linear(64, 1)

        self.reset_parameters()

    def reset_parameters(self):
        self.conv1.reset_parameters()
        self.conv2.reset_parameters()
        self.conv3.reset_parameters()
        self.conv4.reset_parameters()
        self.conv5.reset_parameters()

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.fc0(x))
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = F.relu(self.conv3(x, edge_index))
        x = F.relu(self.conv4(x, edge_index))
        x = F.relu(self.conv5(x, edge_index))
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = torch.squeeze(x, dim=1)
        return x

In [14]:
import time
import torch
import torch.nn.functional as F


def print_info(info):
    message = ('Epoch: {}/{}, Duration: {:.3f}s,'
               'Train Loss: {:.4f}, Test Loss:{:.4f}').format(
                   info['current_epoch'], info['epochs'], info['t_duration'],
                   info['train_loss'], info['test_loss'])
    print(message)


def run(model, train_loader, test_loader, num_nodes, epochs, optimizer, device):
    train_losses = []
    test_losses = []
    for epoch in range(1, epochs + 1):
        t = time.time()
        train_loss = train(model, train_loader, optimizer, device)
        t_duration = time.time() - t
        test_loss = test(model, test_loader, num_nodes, device)
        eval_info = {
            'train_loss': train_loss,
            'test_loss': test_loss,
            'current_epoch': epoch,
            'epochs': epochs,
            't_duration': t_duration
        }

        print_info(eval_info)
        train_losses.append(train_loss)
        test_losses.append(test_loss)
    return train_losses, test_losses



def train(model, train_loader, optimizer, device):
    model.train()

    total_loss = 0
    for idx, data in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data.to(device))
        losses = F.mse_loss(output, data.y.to(device), reduction='none')  # Calculate MSE loss for each item in output and labels
        loss = losses.mean()  # Compute mean loss for backpropagation
        loss.backward(retain_graph=True)
        #loss = model.compute_loss(output, data.y)  # compute loss with L1 regularization
        #loss = F.nll_loss(log_probs, data.y)
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)


def test(model, test_loader, num_nodes, device):
    model.eval()
    correct = 0
    total_loss = 0
    n_graphs = 0
    with torch.no_grad():
        for idx, data in enumerate(test_loader):
            out = model(data.to(device))
            #total_loss += F.nll_loss(out, data.y.to(device)).item()
            total_loss += F.mse_loss(out, data.y.to(device)).item()
            #pred = out.max(1)[1]
            #correct += pred.eq(data.y).sum().item()
            #n_graphs += data.num_graphs
    return total_loss / len(test_loader)
"""
def train(model, train_loader, optimizer, device):
    model.train()

    total_loss = 0
    for idx, data in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data.to(device))
        errors = torch.abs(output - data.y.to(device))
        error_percentage = (errors / (output))
        mean_error_percentage = error_percentage.mean()
        mean_error_percentage.backward(retain_graph=True)
        optimizer.step()
        total_loss += mean_error_percentage
        
    return total_loss / len(train_loader)

def test(model, test_loader, num_nodes, device):
    model.eval()
    correct = 0
    total_loss = 0
    n_graphs = 0
    with torch.no_grad():
        for idx, data in enumerate(test_loader):
            output = model(data.to(device))
            errors = torch.abs(output - data.y.to(device))
            error_percentage = (errors / (output))
            mean_error_percentage = error_percentage.mean()
            total_loss += mean_error_percentage
    return total_loss / len(test_loader)"""

'\ndef train(model, train_loader, optimizer, device):\n    model.train()\n\n    total_loss = 0\n    for idx, data in enumerate(train_loader):\n        optimizer.zero_grad()\n        output = model(data.to(device))\n        errors = torch.abs(output - data.y.to(device))\n        error_percentage = (errors / (output))\n        mean_error_percentage = error_percentage.mean()\n        mean_error_percentage.backward(retain_graph=True)\n        optimizer.step()\n        total_loss += mean_error_percentage\n        \n    return total_loss / len(train_loader)\n\ndef test(model, test_loader, num_nodes, device):\n    model.eval()\n    correct = 0\n    total_loss = 0\n    n_graphs = 0\n    with torch.no_grad():\n        for idx, data in enumerate(test_loader):\n            output = model(data.to(device))\n            errors = torch.abs(output - data.y.to(device))\n            error_percentage = (errors / (output))\n            mean_error_percentage = error_percentage.mean()\n            total_los

In [15]:
#runner
num_nodes = train_loader[0].x.shape[0]
num_features = train_loader[0].x.shape[1]

#leaky_relu
model1 = ArchNN(num_features, num_nodes, heads=8).to(device_name)

optimizer = optim.Adam(model1.parameters(),
                       lr=0.0001)

train_losses, test_losses = \
run(model1, train_loader, test_loader, num_nodes, 100, optimizer, device_name)

#versj defined at the top of the notebook
modelPath = "trained_models\\" + today + "ArchGNNVersion" + str(versj) +".pt"
trainLossPath = "trained_models\\" + today + "ArchGNNVersion" + str(versj) + "TrainLoss.txt"
testLossPath = "trained_models\\" + today + "ArchGNNVersion" + str(versj) + "TestLoss.txt"
np.savetxt(trainLossPath, np.array(train_losses))
np.savetxt(testLossPath, np.load(test_losses))
torch.save(model1.state_dict(), modelPath)
versj += 1
print("saved as: ", modelPath)

Epoch: 1/100, Duration: 17.906s,Train Loss: 4.8944, Test Loss:3.9238
Epoch: 2/100, Duration: 19.364s,Train Loss: 3.2756, Test Loss:3.7240
Epoch: 3/100, Duration: 18.733s,Train Loss: 3.0029, Test Loss:3.2959
Epoch: 4/100, Duration: 22.481s,Train Loss: 2.8255, Test Loss:3.0820
Epoch: 5/100, Duration: 22.077s,Train Loss: 2.7175, Test Loss:3.0337
Epoch: 6/100, Duration: 24.844s,Train Loss: 2.6483, Test Loss:2.7533
Epoch: 7/100, Duration: 21.259s,Train Loss: 2.3036, Test Loss:2.2901
Epoch: 8/100, Duration: 19.854s,Train Loss: 2.0360, Test Loss:1.7537
Epoch: 9/100, Duration: 19.622s,Train Loss: 1.5629, Test Loss:1.3540
Epoch: 10/100, Duration: 20.447s,Train Loss: 1.0485, Test Loss:0.9857
Epoch: 11/100, Duration: 19.644s,Train Loss: 0.6520, Test Loss:0.6751
Epoch: 12/100, Duration: 21.257s,Train Loss: 0.4701, Test Loss:0.4826
Epoch: 13/100, Duration: 24.168s,Train Loss: 0.3915, Test Loss:0.3410
Epoch: 14/100, Duration: 22.079s,Train Loss: 0.3185, Test Loss:0.2541
Epoch: 15/100, Duration: 18.9

TypeError: expected str, bytes or os.PathLike object, not list

In [ ]:

print("saved as: May-19ArchGNNVersion1.pt")

saved as:  May-19ArchGNNVersion1.pt


In [18]:
#runner
num_nodes = train_loader[0].x.shape[0]
num_features = train_loader[0].x.shape[1]

#relu
model2 = ArchNN2(num_features, num_nodes, heads=8).to(device_name)

optimizer = optim.Adam(model2.parameters(),
                       lr=0.0001)

train_losses, test_losses = \
run(model2, train_loader, test_loader, num_nodes, 100, optimizer, device_name)

#versj defined at the top of the notebook
modelPath = "trained_models\\" + today + "ArchGNNReLUVersion" + str(versj) +".pt"
trainLossPath = "trained_models\\" + today + "ArchGNNReLUVersion" + str(versj) + "TrainLoss.txt"
testLossPath = "trained_models\\" + today + "ArchGNNReLUVersion" + str(versj) + "TestLoss.txt"
np.savetxt(trainLossPath, np.array(train_losses))
np.savetxt(testLossPath, np.array(test_losses))
torch.save(model1.state_dict(), modelPath) #this is wrong!! remember after
versj += 1
print("saved as: ", modelPath)

Epoch: 1/100, Duration: 20.105s,Train Loss: 5.4293, Test Loss:4.4863
Epoch: 2/100, Duration: 24.151s,Train Loss: 3.6564, Test Loss:3.7175
Epoch: 3/100, Duration: 21.958s,Train Loss: 3.1565, Test Loss:3.3926
Epoch: 4/100, Duration: 21.947s,Train Loss: 2.9077, Test Loss:3.1633
Epoch: 5/100, Duration: 21.081s,Train Loss: 2.7485, Test Loss:2.9449
Epoch: 6/100, Duration: 19.851s,Train Loss: 2.5428, Test Loss:2.6715
Epoch: 7/100, Duration: 25.160s,Train Loss: 2.1954, Test Loss:2.0479
Epoch: 8/100, Duration: 24.894s,Train Loss: 1.3667, Test Loss:1.0465
Epoch: 9/100, Duration: 27.326s,Train Loss: 0.7322, Test Loss:0.5538
Epoch: 10/100, Duration: 30.844s,Train Loss: 0.4431, Test Loss:0.3601
Epoch: 11/100, Duration: 27.563s,Train Loss: 0.3092, Test Loss:0.2376
Epoch: 12/100, Duration: 26.324s,Train Loss: 0.2445, Test Loss:0.1744
Epoch: 13/100, Duration: 24.663s,Train Loss: 0.1955, Test Loss:0.1299
Epoch: 14/100, Duration: 23.615s,Train Loss: 0.1596, Test Loss:0.0969
Epoch: 15/100, Duration: 21.4

In [15]:
#best one with 1dataset train 4000 and test 1000, Apr-25ver2.pt

class Arch2NN(torch.nn.Module):
    def __init__(self, in_channels, num_classes, heads, t_inv = True):
        super(Arch2NN, self).__init__()
        self.fc0 = nn.Linear(in_channels, 16)
        #self.pool = nn.MaxPool1d(kernel_size=2)  # Add a pooling layer
        self.conv1 = feast_conv.FeaStConv(16, 32, heads=heads, t_inv=t_inv)
        self.conv2 = feast_conv.FeaStConv(32, 64, heads=heads, t_inv=t_inv)
        self.conv3 = feast_conv.FeaStConv(64, 128, heads=heads, t_inv=t_inv)
        self.conv4 = feast_conv.FeaStConv(128, 256, heads=heads, t_inv=t_inv)
        self.conv5 = feast_conv.FeaStConv(256, 512, heads=heads, t_inv=t_inv)
        self.conv6 = feast_conv.FeaStConv(512, 128, heads=heads, t_inv=t_inv)
        #self.conv7 = feast_conv.FeaStConv(256, 128, heads=heads, t_inv=t_inv)
        self.fc1 = nn.Linear(128, 64)
        self.fc2 = nn.Linear(64, 1)

        self.reset_parameters()

    def reset_parameters(self):
        self.conv1.reset_parameters()
        self.conv2.reset_parameters()
        self.conv3.reset_parameters()
        self.conv4.reset_parameters()
        self.conv5.reset_parameters()
        self.conv6.reset_parameters()
        #self.conv7.reset_parameters()

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.leaky_relu(self.fc0(x))
        #x = self.pool(x)
        x = F.leaky_relu(self.conv1(x, edge_index))
        x = F.leaky_relu(self.conv2(x, edge_index))
        x = F.leaky_relu(self.conv3(x, edge_index))
        x = F.leaky_relu(self.conv4(x, edge_index))
        x = F.leaky_relu(self.conv5(x, edge_index))
        x = F.leaky_relu(self.conv6(x, edge_index))
        #x = F.elu(self.conv7(x, edge_index))
        x = F.leaky_relu(self.fc1(x))
        #x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        #F.log_softmax(x, dijm=1)
        x = torch.squeeze(x, dim=1)
        return x


In [112]:
#runner
num_nodes = train_loader[0].x.shape[0]
num_features = train_loader[0].x.shape[1]

model2 = Arch2NN(num_features, num_nodes, heads=8).to(device_name)

optimizer = optim.Adam(model2.parameters(),
                       lr=0.0001)


run(model2, train_loader, test_loader, num_nodes, 300, optimizer, device_name)

#versj defined at the top of the notebook
modelPath = today + "ver" + str(versj) +".pt"
torch.save(model2.state_dict(), modelPath)
versj += 1
print("saved as: ", modelPath)

Epoch: 1/300, Duration: 394.561s,Train Loss: 0.0947, Test Loss:0.0623
Epoch: 2/300, Duration: 400.452s,Train Loss: 0.0221, Test Loss:0.0106
Epoch: 3/300, Duration: 402.195s,Train Loss: 0.0085, Test Loss:0.0039
Epoch: 4/300, Duration: 403.763s,Train Loss: 0.0061, Test Loss:0.0021
Epoch: 5/300, Duration: 370.298s,Train Loss: 0.0045, Test Loss:0.0032
Epoch: 6/300, Duration: 405.726s,Train Loss: 0.0036, Test Loss:0.0030
Epoch: 7/300, Duration: 397.639s,Train Loss: 0.0035, Test Loss:0.0015
Epoch: 8/300, Duration: 430.281s,Train Loss: 0.0030, Test Loss:0.0013
Epoch: 9/300, Duration: 389.513s,Train Loss: 0.0029, Test Loss:0.0010
Epoch: 10/300, Duration: 391.166s,Train Loss: 0.0025, Test Loss:0.0014
Epoch: 11/300, Duration: 399.412s,Train Loss: 0.0024, Test Loss:0.0008
Epoch: 12/300, Duration: 373.854s,Train Loss: 0.0023, Test Loss:0.0006
Epoch: 13/300, Duration: 365.286s,Train Loss: 0.0019, Test Loss:0.0007
Epoch: 14/300, Duration: 366.415s,Train Loss: 0.0020, Test Loss:0.0006
Epoch: 15/300, 

In [16]:
#load model
model5_dict = torch.load("Apr-25ver1.pt")
model6_dict = torch.load("Apr-25ver2.pt")
model7_dict = torch.load("Apr-26ver1.pt")


In [50]:
#torch.save(model5.state_dict(), "Apr-21ver1_state_dict.pt")

In [18]:
num_nodes = train_loader[0].x.shape[0]
num_features = train_loader[0].x.shape[1]

tester_model2 = Arch2NN(num_features, num_nodes, heads=8)
tester_model2.load_state_dict(model6_dict)
tester_model2.eval()

Arch2NN(
  (fc0): Linear(in_features=3, out_features=16, bias=True)
  (conv1): FeaStConv(16, 32, heads=8)
  (conv2): FeaStConv(32, 64, heads=8)
  (conv3): FeaStConv(64, 128, heads=8)
  (conv4): FeaStConv(128, 256, heads=8)
  (conv5): FeaStConv(256, 512, heads=8)
  (conv6): FeaStConv(512, 128, heads=8)
  (fc1): Linear(in_features=128, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=1, bias=True)
)

In [22]:
tester_model1 = ArchNN(num_features, num_nodes, heads=8)

tester_model3 = Arch2NN(num_features, num_nodes, heads=8)
tester_model1.load_state_dict(model5_dict)
tester_model3.load_state_dict(model7_dict)
tester_model1.eval()
tester_model3.eval()
tester_model3.share_memory

<bound method Module.share_memory of Arch2NN(
  (fc0): Linear(in_features=3, out_features=16, bias=True)
  (conv1): FeaStConv(16, 32, heads=8)
  (conv2): FeaStConv(32, 64, heads=8)
  (conv3): FeaStConv(64, 128, heads=8)
  (conv4): FeaStConv(128, 256, heads=8)
  (conv5): FeaStConv(256, 512, heads=8)
  (conv6): FeaStConv(512, 128, heads=8)
  (fc1): Linear(in_features=128, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=1, bias=True)
)>

In [129]:
tester_model1.eval()
print(tester_model1(datasetTest[200]))
print(datasetTest[200].y)

tester_model2.eval()
print(tester_model2(datasetTest[200]))
print(datasetTest[200].y)

tester_model3.eval()
print(tester_model3(datasetTest2[0]))
print(datasetTest2[0].y)

tensor([-3.1303,  2.3542,  1.5644,  0.8224,  0.1171, -0.4801, -1.0095, -1.4173,
        -1.6761, -1.8224, -1.8244, -1.6886, -1.3957, -1.0161, -0.4973,  0.1045,
         0.8055,  1.5466,  2.3431,  3.1168], grad_fn=<SqueezeBackward1>)
tensor([-3.1242,  2.3410,  1.5610,  0.8124,  0.1205, -0.4919, -1.0057, -1.4050,
        -1.6780, -1.8165, -1.8165, -1.6780, -1.4050, -1.0057, -0.4919,  0.1205,
         0.8124,  1.5610,  2.3410,  3.1242])
tensor([-3.1318,  2.3414,  1.5673,  0.8168,  0.1115, -0.5086, -1.0223, -1.4418,
        -1.6993, -1.8431, -1.8433, -1.7036, -1.4348, -1.0281, -0.5020,  0.1201,
         0.8166,  1.5717,  2.3660,  3.1554], grad_fn=<SqueezeBackward1>)
tensor([-3.1242,  2.3410,  1.5610,  0.8124,  0.1205, -0.4919, -1.0057, -1.4050,
        -1.6780, -1.8165, -1.8165, -1.6780, -1.4050, -1.0057, -0.4919,  0.1205,
         0.8124,  1.5610,  2.3410,  3.1242])
tensor([ 2.6688, -0.6248,  0.6508,  1.2117,  1.2339,  0.9167,  0.2930, -0.3265,
        -0.8476, -1.1294, -1.1046, -0.8399, 

In [130]:
testFinal(tester_model3, datasetTest2, device_name)

[-943.92, -970.89, -948.52, -987.8, -997.61, -943.48, -980.23, -1039.57, -1007.18, -1011.52, -1036.51, -1017.27, -1062.31, -953.94, -945.2, -945.05, -914.52, -1019.61, -1104.04, -1066.3]
Avarage Error: 994.77%, Adjusted Avarage Error: 984.73%
Mean Square: 130.8571

[-951.31, -977.89, -960.97, -1002.53, -1007.59, -948.62, -987.03, -1057.16, -1021.67, -1031.18, -1061.54, -1037.75, -1086.54, -966.99, -947.21, -944.99, -910.13, -1024.24, -1127.92, -1083.64]
Avarage Error: 1006.85%, Adjusted Avarage Error: 995.69%
Mean Square: 139.2679

[-959.2, -987.29, -973.06, -1017.36, -1018.11, -954.3, -994.52, -1074.95, -1035.72, -1046.5, -1077.21, -1060.06, -1113.77, -982.0, -946.93, -943.89, -904.25, -1023.76, -1158.22, -1100.32]
Avarage Error: 1018.57%, Adjusted Avarage Error: 1005.52%
Mean Square: 148.0589

[-967.55, -997.65, -986.45, -1032.59, -1029.2, -959.43, -1003.24, -1091.49, -1050.67, -1064.13, -1095.75, -1085.83, -1140.44, -996.17, -948.28, -940.86, -893.69, -1020.78, -1189.63, -1116.28]
A

In [104]:
testFinal(tester_model2, datasetNew4, device_name)

[86.29, 85.38, 88.6, 88.33, 86.96, 86.97, 154.74, 94.72, 89.06, 395.3, 81.07, 160.95, 95.86, 85.02, 84.8, 84.81, 78.52, 75.03, 86.72, 82.46]
Avarage Error: 108.58%, Adjusted Avarage Error: 89.74%
Mean Square: 25.618

[86.2, 85.26, 88.57, 88.28, 86.89, 86.89, 153.49, 94.72, 88.98, 396.95, 80.93, 159.33, 95.83, 84.8, 84.65, 84.73, 78.19, 74.29, 86.8, 82.3]
Avarage Error: 108.41%, Adjusted Avarage Error: 89.55%
Mean Square: 25.9762

[86.12, 85.15, 88.54, 88.24, 86.82, 86.8, 152.38, 94.7, 88.91, 399.48, 80.8, 158.18, 95.81, 84.57, 84.51, 84.66, 77.91, 73.55, 86.87, 82.15]
Avarage Error: 108.31%, Adjusted Avarage Error: 89.36%
Mean Square: 26.3286

[86.03, 85.04, 88.51, 88.19, 86.75, 86.71, 151.46, 94.66, 88.83, 405.11, 80.67, 157.22, 95.78, 84.29, 84.37, 84.59, 77.65, 72.77, 86.98, 81.99]
Avarage Error: 108.38%, Adjusted Avarage Error: 89.18%
Mean Square: 26.6836

[85.95, 84.92, 88.47, 88.14, 86.68, 86.62, 150.64, 94.62, 88.75, 411.5, 80.53, 156.41, 95.75, 83.98, 84.24, 84.52, 77.42, 71.93

In [71]:
testData(tester_model, datasetNew,1)

real      out       %error    Sign      
4.303     4.305     0.06      True      
-0.844    -0.849    0.54      True      
1.293     1.291     -0.16     True      
2.235     2.237     0.09      True      
2.203     2.202     -0.05     True      
1.485     1.481     -0.24     True      
0.398     0.399     0.43      True      
-0.747    -0.754    1.03      True      
-1.677    -1.681    0.21      True      
-2.195    -2.203    0.36      True      
-2.195    -2.206    0.49      True      
-1.677    -1.692    0.87      True      
-0.747    -0.766    2.58      True      
0.398     0.397     -0.11     True      
1.485     1.493     0.56      True      
2.203     2.216     0.56      True      
2.235     2.248     0.55      True      
1.293     1.294     0.06      True      
-0.844    -0.857    1.48      True      
-4.303    -4.320    0.39      True      


In [195]:
# printing parameters

def count_parameters(model):
    """Count the number of trainable parameters in a PyTorch model."""
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


# Print the number of parameters
print("Number of parameters in the model: {}".format(count_parameters(model4)))

Number of parameters in the model: 95545
